In [14]:
import os

import numpy as np

from Architectures.OptimalCNN import OptimalCNN
from DataObjects import DataLoader
import torch.optim as optim
from Architectures.StochasticDepthCNN import StochasticDepthCNN
from utils import *
import matplotlib.pyplot as plt

from typing import Optional, Tuple

In [15]:
def train_model(model: nn.Module, train_loader: DataLoader, val_loader: DataLoader,
                num_epochs: int = 10, lr: float = 0.001,
                device: torch.device = None) -> None:
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion: nn.Module = nn.CrossEntropyLoss()
    optimizer: torch.optim.Optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        model.train()
        train_loss: float = 0.0
        train_correct: int = 0
        total_train: int = 0

        for batch in train_loader:
            inputs = batch.data.to(device)
            labels = batch.labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += torch.sum(preds == labels).item()
            total_train += labels.size(0)

        avg_train_loss = train_loss / total_train
        train_acc = train_correct / total_train

        model.eval()
        val_loss: float = 0.0
        val_correct: int = 0
        total_val: int = 0

        with torch.no_grad():
            for batch in val_loader:
                inputs = batch.data.to(device)
                labels = batch.labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += torch.sum(preds == labels).item()
                total_val += labels.size(0)

        avg_val_loss = val_loss / total_val
        val_acc = val_correct / total_val

        print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {avg_train_loss:.4f}, Train acc: {train_acc:.4f} | Val loss: {avg_val_loss:.4f}, Val acc: {val_acc:.4f}")

def evaluate(model: nn.Module,
             test_loader: Optional[DataLoader] = None,
             device: Optional[torch.device] = None) -> Tuple[float, float]:

    if test_loader is None:
        test_dir = os.path.join("Data", "Data_converted", "test")
        test_loader = DataLoader(test_dir, batch_size=64, shuffle=True)

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    criterion = nn.CrossEntropyLoss()
    model.eval()

    test_loss = 0.0
    test_correct = 0
    total_test = 0

    with torch.no_grad():
        for batch in test_loader:
            inputs = batch.data.to(device)
            labels = batch.labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            test_correct += torch.sum(preds == labels).item()
            total_test += labels.size(0)

    avg_test_loss = test_loss / total_test
    test_acc = test_correct / total_test

    print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_acc:.4f}")
    return avg_test_loss, test_acc

In [16]:
# Setup directories and DataLoaders
train_dir_10 = os.path.join("Data", "Data_few_10", "train")
train_dir_15 = os.path.join("Data", "Data_few_15", "train")
train_dir_20 = os.path.join("Data", "Data_few_20", "train")

val_dir_10 = os.path.join("Data", "Data_few_10", "valid")
val_dir_15 = os.path.join("Data", "Data_few_15", "valid")
val_dir_20 = os.path.join("Data", "Data_few_20", "valid")

test_dir_10 = os.path.join("Data", "Data_converted", "test")
test_dir_15 = os.path.join("Data", "Data_converted", "test")
test_dir_20 = os.path.join("Data", "Data_converted", "test")

train_loader_10 = DataLoader(train_dir_10, batch_size=32, shuffle=True, max_per_class = 3000)
train_loader_15 = DataLoader(train_dir_15, batch_size=32, shuffle=True, max_per_class = 3000)
train_loader_20 = DataLoader(train_dir_20, batch_size=32, shuffle=True, max_per_class = 3000)

val_loader_10 = DataLoader(val_dir_10, batch_size=32, shuffle=True, max_per_class = 3000)
val_loader_15 = DataLoader(val_dir_15, batch_size=32, shuffle=True, max_per_class = 3000)
val_loader_20 = DataLoader(val_dir_20, batch_size=32, shuffle=True, max_per_class = 3000)

test_loader_10 = DataLoader(test_dir_10, batch_size=32, shuffle=True, max_per_class = 3000)
test_loader_15 = DataLoader(test_dir_15, batch_size=32, shuffle=True, max_per_class = 3000)
test_loader_20 = DataLoader(test_dir_20, batch_size=32, shuffle=True, max_per_class = 3000)

In [17]:
model_10 = load_model("/home/piotr/PycharmProjects/dl_project1/DeepLearning/Project_I/Models_Pytorch_saved/model_Optimal_I_trained_saved.pth")
# Train the model
train_model(model_10, train_loader_10, val_loader_10, num_epochs=10, lr=0.008)
# Evaluate the model
evaluate(model_10, test_loader_10)
# Save the model
save_model(model_10, "Models_Pytorch_saved/model_few_10_trained_saved.pth")

Model loaded successfully from /home/piotr/PycharmProjects/dl_project1/DeepLearning/Project_I/Models_Pytorch_saved/model_Optimal_I_trained_saved.pth
Epoch 1/10 - Train loss: 1.1123, Train acc: 0.6459 | Val loss: 1.1845, Val acc: 0.6088
Epoch 2/10 - Train loss: 0.6885, Train acc: 0.7650 | Val loss: 1.2272, Val acc: 0.6200
Epoch 3/10 - Train loss: 0.4362, Train acc: 0.8513 | Val loss: 1.6295, Val acc: 0.5914
Epoch 4/10 - Train loss: 0.2968, Train acc: 0.9027 | Val loss: 1.6904, Val acc: 0.5738
Epoch 5/10 - Train loss: 0.2063, Train acc: 0.9290 | Val loss: 1.7008, Val acc: 0.6074
Epoch 6/10 - Train loss: 0.1873, Train acc: 0.9357 | Val loss: 1.8893, Val acc: 0.5962
Epoch 7/10 - Train loss: 0.1630, Train acc: 0.9456 | Val loss: 2.0453, Val acc: 0.5831
Epoch 8/10 - Train loss: 0.1495, Train acc: 0.9502 | Val loss: 2.2173, Val acc: 0.5894
Epoch 9/10 - Train loss: 0.1214, Train acc: 0.9591 | Val loss: 2.0586, Val acc: 0.5900
Epoch 10/10 - Train loss: 0.1212, Train acc: 0.9590 | Val loss: 2.24

In [18]:
model_15 = load_model("/home/piotr/PycharmProjects/dl_project1/DeepLearning/Project_I/Models_Pytorch_saved/model_Optimal_I_trained_saved.pth")
# Train the model
train_model(model_15, train_loader_15, val_loader_15, num_epochs=10, lr=0.008)
# Evaluate the model
evaluate(model_15, test_loader_15)
# Save the model
save_model(model_15, "Models_Pytorch_saved/model_few_15_trained_saved.pth")

Model loaded successfully from /home/piotr/PycharmProjects/dl_project1/DeepLearning/Project_I/Models_Pytorch_saved/model_Optimal_I_trained_saved.pth
Epoch 1/10 - Train loss: 1.0689, Train acc: 0.6601 | Val loss: 1.0879, Val acc: 0.6143
Epoch 2/10 - Train loss: 0.7250, Train acc: 0.7548 | Val loss: 1.2834, Val acc: 0.6139
Epoch 3/10 - Train loss: 0.4998, Train acc: 0.8269 | Val loss: 1.3639, Val acc: 0.6109
Epoch 4/10 - Train loss: 0.3545, Train acc: 0.8746 | Val loss: 1.4921, Val acc: 0.6067
Epoch 5/10 - Train loss: 0.2875, Train acc: 0.9006 | Val loss: 1.7295, Val acc: 0.5955
Epoch 6/10 - Train loss: 0.2090, Train acc: 0.9313 | Val loss: 1.8322, Val acc: 0.5953
Epoch 7/10 - Train loss: 0.1915, Train acc: 0.9341 | Val loss: 1.8612, Val acc: 0.5993
Epoch 8/10 - Train loss: 0.1628, Train acc: 0.9456 | Val loss: 2.0281, Val acc: 0.6039
Epoch 9/10 - Train loss: 0.1554, Train acc: 0.9473 | Val loss: 2.0280, Val acc: 0.5907
Epoch 10/10 - Train loss: 0.1496, Train acc: 0.9492 | Val loss: 2.16

In [19]:
model_20 = load_model("/home/piotr/PycharmProjects/dl_project1/DeepLearning/Project_I/Models_Pytorch_saved/model_Optimal_I_trained_saved.pth")
# Train the model
train_model(model_20, train_loader_20, val_loader_20, num_epochs=10, lr=0.008)
# Evaluate the model
evaluate(model_20, test_loader_20)
# Save the model
save_model(model_20, "Models_Pytorch_saved/model_few_20_trained_saved.pth")

Model loaded successfully from /home/piotr/PycharmProjects/dl_project1/DeepLearning/Project_I/Models_Pytorch_saved/model_Optimal_I_trained_saved.pth
Epoch 1/10 - Train loss: 1.0709, Train acc: 0.6556 | Val loss: 1.0743, Val acc: 0.6254
Epoch 2/10 - Train loss: 0.7660, Train acc: 0.7366 | Val loss: 1.1496, Val acc: 0.6184
Epoch 3/10 - Train loss: 0.5609, Train acc: 0.8035 | Val loss: 1.3112, Val acc: 0.6174
Epoch 4/10 - Train loss: 0.4104, Train acc: 0.8556 | Val loss: 1.3273, Val acc: 0.6092
Epoch 5/10 - Train loss: 0.3117, Train acc: 0.8934 | Val loss: 1.6623, Val acc: 0.6180
Epoch 6/10 - Train loss: 0.2418, Train acc: 0.9172 | Val loss: 1.7521, Val acc: 0.6158
Epoch 7/10 - Train loss: 0.2107, Train acc: 0.9260 | Val loss: 1.9155, Val acc: 0.5843
Epoch 8/10 - Train loss: 0.1913, Train acc: 0.9334 | Val loss: 1.8574, Val acc: 0.6093
Epoch 9/10 - Train loss: 0.1659, Train acc: 0.9439 | Val loss: 2.0242, Val acc: 0.5984
Epoch 10/10 - Train loss: 0.1602, Train acc: 0.9468 | Val loss: 2.07